# Error Handling

## Python Errors (Exceptions)

In [ ]:
my_list = [1, 2, 3, "Four", 5, 6, 7]

In [ ]:
for element in my_list:
    result = 10 + element
    print(result)

## Try and except

In [ ]:
my_list = [1, 2, 3, "Four", 5, 6, 7]

In [ ]:
for element in my_list:
    try:
        result = 10 + element
        print(result)
    except:
        print("{} is not a number.".format(element))

## Catching specific Errors

In [ ]:
my_list = [1, 2, 3, "Four", 5, 6, 7]

In [ ]:
for element in my_list:
    try:
        result = 10 + element
        print(result)
    except TypeError: # only TypeErrors are handled
        print("{} is not a number.".format(element))

In [ ]:
for element in my_list:
    try:
        result = 10 + element
        print(result)
    except ValueError: # only ValueErrors are handled
        print("{} is not a number.".format(element))

## The Exception class

In [ ]:
my_list = [1, 2, 3, "Four", 5, 6, 7]

In [ ]:
for element in my_list:
    try:
        result = 10 + element
        print(result)
    except Exception as e:
        print("{} is not a number".format(element), end = " | ")
        print(e)

## try, except, else

In [ ]:
my_list = [1, 2, 3, "Four", 5, 6, 7]

In [ ]:
for element in my_list:
    try:
        result = 10 + element
    except Exception as e:
        print("{} is not a number".format(element),  end = " | ")
        print(e)
    else: # only if no error occured
        print(result, end = " | ")
        print("{} is a valid input".format(element))

In [ ]:
# isn´t this identical? no!
for element in my_list:
    try:
        result = 10 + element
        print(result, end = " | ") # catches unexpected errors in this line as well
        print("{} is a valid input".format(element)) # catches unexpected errors in this line as well
    except Exception as e:
        print("{} is not a number".format(element),  end = " | ")
        print(e)

## finally

In [ ]:
my_list = [1, 2, 3, "Four", 5, 6, 7]

In [ ]:
for element in my_list:
    try:
        result = 10 + element
    except Exception as e:
        print("{} is not a number.".format(element), end = " | ")
        print(e, end = " | ")
    else: # only if no error occured
        print(result, end = " | ")
        print("{} is a valid input".format(element), end = " | ")
    finally: # in any case
        print("Valid or not valid, it doesn´t matter!")

## Try again (...until it works)

In [ ]:
import numpy as np

In [ ]:
# process with random outcome (0: No Success, 1: Success)
np.random.randint(0, 2)

In [ ]:
1 / 0 # if 0: Error

In [ ]:
result = 1 / np.random.randint(0, 2)
print(result)

Goal: Re-try until it works (success)

In [ ]:
attempt = 0
while True:
    try:
        result = 1 / np.random.randint(0, 2)
    except Exception as e:
        print(e, end = " | ")
    else: 
        print(result, end = " | ")
        break # stop when it worked
    finally:
        attempt += 1
        print("Attempt: {}".format(attempt))

## Limited number of retries

__Stop after [3] unsuccessful attempts.__

In [ ]:
import numpy as np

In [ ]:
max_attempts = 3

In [ ]:
attempt = 0
success = False
while True:
    try:
        result = 1 / np.random.randint(0, 2)
    except Exception as e:
        print(e, end = " | ")
    else: 
        print(result, end = " | ")
        success = True
        break
    finally:
        attempt += 1
        print("Attempt: {}".format(attempt))
        if success == False:
            if attempt >= max_attempts: # if max_attempts is reached
                print("Programm stopped: max_attempts reached!")
                break

## Waiting periods between re-tries

In [ ]:
import numpy as np
import time

In [ ]:
attempt = 0
max_attempts = 4
success = False
wait = 1 # initial waiting period: 1 second
wait_increase = 5 # waiting period increase in seconds

while True:
    try:
        result = 1 / np.random.randint(0, 2)
    except Exception as e:
        print(e, end = " | ")
    else: 
        print(result, end = " | ")
        success = True
        break
    finally:
        attempt += 1
        print("Attempt: {}".format(attempt))
        if success == False:
            if attempt >= max_attempts: # if max_attempts is reached
                print("Programm stopped: max_attempts reached!")
                break
            else: # if we haven´t reached max_attempts, wait before we start the next try
                time.sleep(wait)
                wait += wait_increase # increase waiting time

## Error Handling Application: Trading with IBKR

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with an IBKR Practice/Demo Account and NOT with a Live Trading Account._

__What can go wrong?__
- API Calls (mostly handled with try/except)
- Connectivity Issues: IBKR very realiable, connect. breaks do not stop the whole session (but the stream...) 

### Recap: Contrader with start and stop wrapped into functions

In [ ]:
from ib_async import * # now use ib_async 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop()

In [ ]:
def start_session(): # NEW
    global session_start
    
    session_start = pd.to_datetime(datetime.now(timezone.utc))# new (Python 3.12)
    
    initialize_stream() # NEW 
    stop_session() # NEW

def initialize_stream(): # NEW
    global bars, last_bar
    
    bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting=freq,
            whatToShow='MIDPOINT',
            useRTH=True,
            formatDate=2,
            keepUpToDate=True)
    last_bar = bars[-1].date
    bars.updateEvent += onBarUpdate 

def onBarUpdate(bars, hasNewBar):  
    global df, last_bar
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())
        df["position"] = -np.sign(df.returns.rolling(window).mean())
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)

def stop_session(): # NEW
    while True:
        ib.sleep(5) # check every 5 seconds
        if datetime.now(timezone.utc).time() >= end_time: # if stop conditions has been met
            execute_trade(target = 0) # close open position 
            ib.cancelHistoricalData(bars) # stop stream
            ib.sleep(10)
            try:
                trade_reporting() # final reporting
            except:
                pass
            print("Session Stopped.")
            ib.disconnect()
            break
        else:
            pass

In [ ]:
ib = IB()
ib.connect()

In [ ]:
# strategy parameters
freq = "1 min"
window = 1
units = 1000
end_time = (datetime.now(timezone.utc) + dt.timedelta(seconds = 330)).time() # stop condition (5.5 mins from now)
print(end_time)
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [ ]:
start_session()

## Error Handling: IBKR API Connectivity Issues

In [ ]:
from ib_async import * 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone # new
from IPython.display import display, clear_output
util.startLoop()

In [ ]:
def start_session():
    global last_update, session_start
    
    last_update = datetime.now(timezone.utc) # NEW 
    session_start = pd.to_datetime(last_update) # Updated (Python 3.12)
    
    initialize_stream()  
    stop_session()

def initialize_stream(): 
    global bars, last_bar
    
    bars = ib.reqHistoricalData(
            contract,
            endDateTime='',
            durationStr='1 D',
            barSizeSetting=freq,
            whatToShow='MIDPOINT',
            useRTH=True,
            formatDate=2,
            keepUpToDate=True)
    last_bar = bars[-1].date
    bars.updateEvent += onBarUpdate 
    
def onBarUpdate(bars, hasNewBar):  
    global df, last_bar, last_update
    
    last_update = datetime.now(timezone.utc) # NEW
    
    if bars[-1].date > last_bar: 
        last_bar = bars[-1].date
    
        # Data Processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1] 
        df.set_index("date", inplace = True)
        
        ####################### Trading Strategy ###########################
        df = df[["close"]].copy()
        df["returns"] = np.log(df["close"] / df["close"].shift())
        df["position"] = -np.sign(df.returns.rolling(window).mean())
        ####################################################################
        
        # Trading
        target = df["position"][-1] * units
        execute_trade(target = target)
        
        # Display
        clear_output(wait=True)
        display(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    
    # 1. get current Position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
         
    # 2. identify required trades
    trades = target - current_pos
        
    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)  
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report
    
    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis = 1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side":"first", "cumQty":"max", "avgPrice":"mean", "realizedPNL":"sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()
        
    clear_output(wait=True)
    display(df, report)

def stop_session():
    while True:
        ib.sleep(5) 
        if datetime.now(timezone.utc).time() >= end_time:
            execute_trade(target = 0) 
            ib.cancelHistoricalData(bars) 
            ib.sleep(10)
            try:
                trade_reporting() 
            except:
                pass
            print("Session Stopped (planned).")
            ib.disconnect()
            break
        elif datetime.now(timezone.utc) - last_update > dt.timedelta(seconds=120):
                # if there was no streaming response in the last 120 seconds
                ib.cancelHistoricalData(bars)
                ib.sleep(5)
                try: # try to reestablish stream
                    initialize_stream() # one retry
                except: # stop session
                    ib.sleep(5)
                    try:
                        execute_trade(target = 0) # close open position
                    except:
                        pass
                    ib.sleep(10)
                    try:
                        trade_reporting() # final reporting
                    except:
                        pass
                    print("Session Stopped - No Connection.")
                    ib.disconnect()
                    break
        else:
            pass

In [ ]:
ib = IB()
ib.connect()

In [ ]:
# strategy parameters
freq = "1 min"
window = 1
units = 1000
end_time = (datetime.now(timezone.utc) + dt.timedelta(seconds = 330)).time() # stop condition (5.5 mins from now)
contract = Forex('EURUSD') 
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency = "USD")
ib.qualifyContracts(cfd)
conID = cfd.conId

In [ ]:
start_session()